**1. 준비 (GPU T4로 세팅)**

In [ ]:
!nvidia-smi -L || echo "No GPU detected (ok for quick tests)"
!pip -q install "transformers" "datasets==3.0.1" "torch==2.3.0+cu118" "torchvision==0.18.0+cu118" "torchaudio==2.3.0+cu118" "sentencepiece" \
                 "pandas" "numpy" "matplotlib" "tqdm" "evaluate" -f https://download.pytorch.org/whl/torch_stable.html

GPU 0: Tesla T4 (UUID: GPU-ca0fd2a3-bb83-7973-5e49-fa7d4c166ea1)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.6/839.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 M

**2. Github Link 추가하여 데이터 불러오기, 데이터 파일 명만 추가하여 파일 구분**

In [ ]:
import pandas as pd

# GitHub raw URL 목록
urls = [
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/1.R1%7E2.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/2.RTM.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/3.R3%7E5.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/4.PLAY-IN.csv",
    "https://raw.githubusercontent.com/ellastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/5.PLAYOFF.csv"
]

dfs = []

for url in urls:
    # 파일 이름만 추출 (예: Round1_Bahrain.csv → Round1_Bahrain)
    filename = url.split("/")[-1].replace(".csv", "")

    # CSV 읽기
    df = pd.read_csv(url)

    dfs.append(df)

# 모든 데이터 합치기
df_all = pd.concat(dfs, ignore_index=True)

In [ ]:
df_all.head()

,time_text,timestamp,author,message,amount,format,date
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,NaN,R1~2,250604
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,NaN,R1~2,250604
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,NaN,R1~2,250604
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,NaN,R1~2,250604
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,NaN,R1~2,250604


**3. 데이터 컬럼 선택 및 클리닝**

클리닝
- 디시인사이드의 경우 "dc official App" 삭제
- 해시태그, 멘션, URL, 공백 삭제
- "ㅋㅋㅋ"나 "!!!" 등의 표현 수는 남겨두기

**추후논의하여 클리닝 수정 가능

In [ ]:
# 마이크로초 단위 timestamp를 datetime 객체로 변환
# unit='us'를 사용하여 마이크로초 단위임을 지정합니다.
df_all['datetime_utc'] = pd.to_datetime(df_all['timestamp'], unit='us', errors='coerce')

# 초 이하 단위 제거
df_all['datetime_utc'] = df_all['datetime_utc'].dt.floor('s')

# UTC 시간을 KST (UTC+9)로 변환
# 먼저 UTC 타임존 정보를 추가하고, 그 다음 KST로 변환합니다.
df_all['datetime_kst'] = df_all['datetime_utc'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')

# 초 이하 단위 제거
df_all['datetime_kst'] = df_all['datetime_kst'].dt.floor('s')

In [ ]:
#결과 확인
df_all.head()

,time_text,timestamp,author,message,amount,format,date,datetime_utc,datetime_kst
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,NaN,R1~2,250604,2025-06-04 05:20:50,2025-06-04 14:20:50+09:00
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,NaN,R1~2,250604,2025-06-04 05:32:59,2025-06-04 14:32:59+09:00
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,NaN,R1~2,250604,2025-06-04 05:42:30,2025-06-04 14:42:30+09:00
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,NaN,R1~2,250604,2025-06-04 05:49:39,2025-06-04 14:49:39+09:00
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,NaN,R1~2,250604,2025-06-04 05:49:57,2025-06-04 14:49:57+09:00


In [ ]:
import pandas as pd
import numpy as np
import re

# ---------------------------------------------------------------------
# 데이터 클리닝

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""

    # 1) URL 제거
    s = re.sub(r"http\S+|www\.\S+", " ", s)

    # 2) 멘션/해시태그 제거
    s = re.sub(r"[@#]\w+", " ", s)

    # 3) 반복 문자 축약 (ㅋㅋㅋㅋ -> ㅋㅋ, 아아아아 -> 아아, !!!!! -> !!)
    s = re.sub(r"([ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9!?.])\1{2,}", r"\1\1", s)

    #4) 이모지/특수문자 과도한 것 정리 (한글/영문/숫자/기본문장부호만 유지)
    s = re.sub(r"[^\w\s가-힣ㄱ-ㅎㅏ-ㅣ!?.]", " ", s, flags=re.UNICODE)

    # 생략 5) 팀명/드라이버 약칭 정규화 (원하면 계속 추가)
    # team_map = {
    #    "T1": "티원",
    #    "GEN": "젠지",
    #    "VER": "페르스타펜",
    #    "HAM": "해밀턴",
    #}
    #for short_name, full_name in team_map.items():
    #    s = re.sub(fr"\b{short_name}\b", full_name, s, flags=re.IGNORECASE)

    # 6) 공백 정리
    s = re.sub(r"\s+", " ", s).strip()
    return s

####클리닝은 논의 후 추가

# ---------------------------------------------------------------------
print("Before cleaning, rows:", len(df_all))
# 데이터프레임 컬럼 선택 (df_all에서 필요한 컬럼 선택)
df = df_all[["datetime_kst", "author", "message", "format"]].copy()

# 채팅 제한 봇 데이터 삭제 (유저id: UCSvjQBDgYDB5TGVmCZObcwA)
df = df[df['author'] != 'UCSvjQBDgYDB5TGVmCZObcwA'].copy()

# 클리닝
df["clean_text"] = df["message"].map(normalize_text)


# 빈 텍스트/공백만 남은 행 제거
df["clean_text"] = df["clean_text"].fillna("").str.strip()
df = df[df["clean_text"].astype(bool)].reset_index(drop=True)

print("After cleaning, rows:", len(df))


Before cleaning, rows: 1520294
After cleaning, rows: 1504522


In [ ]:
sample = df.sample(100)
sample = sample['clean_text']
sample.to_csv("LCK_sample_100.csv", index=False, encoding='euc-kr')

In [ ]:
sample.head(20)

,time_text,timestamp,author,message,amount,format,date,datetime_utc,datetime_kst
1164635,5:39:47,1753614607974689,UChRJ9tSC-4wZr-btK9qFjhw,대 리 아,NaN,R3~5,250727,2025-07-27 11:10:07,2025-07-27 20:10:07+09:00
1280352,1:24:58,1758436829249708,UCZl2C-3v9Y5YoxT8QI2M0eQ,ㅇㅇ,NaN,PLAYOFF,250921,2025-09-21 06:40:29,2025-09-21 15:40:29+09:00
1293377,2:46:51,1758441742333389,UCh3V6q804PJHu0Ozn0hwCcw,밴픽왜이래 ㅋㅋ,NaN,PLAYOFF,250921,2025-09-21 08:02:22,2025-09-21 17:02:22+09:00
1386519,1:05:31,1757831747494658,UCTTz391eHJQg42vYl2zPMWw,용 안뺏겼으면 4용이네 ㄲㅂ,NaN,PLAYOFF,250914,2025-09-14 06:35:47,2025-09-14 15:35:47+09:00
993028,1:52:29,1755162285140682,UCbcney2Y9K1OflzA3gaN55A,어우.. 보기가 힘드네,NaN,R3~5,250814,2025-08-14 09:04:45,2025-08-14 18:04:45+09:00
536725,2:37:39,1744531700551129,UC58-yEncHGCJboi8fodODMA,이게 템플릿임,NaN,R1~2,250413,2025-04-13 08:08:20,2025-04-13 17:08:20+09:00
1107417,3:35:42,1754124963004893,UCR_v7uPmstPo7_25B95m70w,럼블궁 답없네..,NaN,R3~5,250802,2025-08-02 08:56:03,2025-08-02 17:56:03+09:00
1505321,5:18:01,1759051102767801,UCjbmROgYQI1a8XOgZD3KYYw,막연하게 이길거같다 ㄷㄷ,NaN,PLAYOFF,250928,2025-09-28 09:18:22,2025-09-28 18:18:22+09:00
882095,3:06:13,1756024593870526,UCT9HAbEU74DwK42-WzFXqvw,왜 구마유시만 골드가 밀리나요?,NaN,R3~5,250824,2025-08-24 08:36:33,2025-08-24 17:36:33+09:00
5228,1:56:56,1749028347434407,UCWfV3R4MfMn_wDNW2AJRTsg,페이커 스킨 ㄷㄷ,NaN,R1~2,250604,2025-06-04 09:12:27,2025-06-04 18:12:27+09:00


In [ ]:
#데이터 확인
df.sample(5)

,datetime_kst,author,message,format,clean_text
1461601,2025-09-10 21:22:26+09:00,UCh7Z-WIQi_zjvKJE0XSli2A,대리아,PLAYOFF,대리아
756033,2025-06-13 19:10:09+09:00,UCC14QhgMZzqZJ201FrywEEg,셰셰!!!!!,RTM,셰셰!!
415830,2025-04-25 20:46:00+09:00,UCg-NM5NOb4HJZeaoBl4MkAA,도란아 보여주자 그라가스 가자,R1~2,도란아 보여주자 그라가스 가자
84507,2025-05-25 20:16:31+09:00,UCtoXKEdR0gzAD8u8I2RSr9A,방금 한정 데상혁이네,R1~2,방금 한정 데상혁이네
322828,2025-05-04 18:59:29+09:00,UCidWwf4ou61EIYgHVtn7JAA,두두 두드 ㄷㄷ,R1~2,두두 두드 ㄷㄷ


**4. 파인튜닝을 위한 데이터셋 불러오기**

사용데이터: KMRE
- KMRE: https://github.com/passing2961/KMRE


In [ ]:
import pickle
import requests

base_url = 'https://raw.githubusercontent.com/passing2961/KMRE/master/data/'

# Train 데이터
kmre_train_data = pickle.loads(requests.get(base_url + 'kmre_train').content)

# Dev 데이터
kmre_dev_data = pickle.loads(requests.get(base_url + 'kmre_dev').content)

# Test 데이터
kmre_test_data = pickle.loads(requests.get(base_url + 'kmre_test').content)

print(f"Train: {len(kmre_train_data)}, Dev: {len(kmre_dev_data)}, Test: {len(kmre_test_data)}")

Train: 119995, Dev: 29999, Test: 49997


In [ ]:
#            0         1         2        3          4            5
labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']

In [ ]:
kmre_train_data[0:5]

[['너~~~무 재미없어서 중간에 나왔습니다;;', np.int32(2)],
 ['10대를위한미드..김수현풍,출생의비밀,그물망애정라인', np.int32(4)],
 ['개발암 .... 이기적인 새끼들', np.int32(0)],
 ['지루해 죽는줄 알았네 보지마', np.int32(1)],
 ['이게 10점이라고? 절대보지마라', np.int32(2)]]

**5. KcELECTRA 모델 불러오기**

In [ ]:
# 1. 토크나이저 및 모델, 데이터셋 준비
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset # Import Dataset class
import torch
import numpy as np # numpy import 추가

tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=128)

# Convert kmre data to Hugging Face Dataset format
def create_dataset(data_list, labels):
    texts = [item[0] for item in data_list]
    # Ensure labels are integers for classification
    numeric_labels = [item[1].item() if isinstance(item[1], np.ndarray) else int(item[1]) for item in data_list]
    return Dataset.from_dict({'text': texts, 'label': numeric_labels})

raw_train_dataset = create_dataset(kmre_train_data, labels)
raw_eval_dataset = create_dataset(kmre_test_data, labels)

# Apply preprocessing (tokenization)
tokenized_train_dataset = raw_train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = raw_eval_dataset.map(preprocess_function, batched=True)

# 2. 라벨/멀티라벨 설정
model = AutoModelForSequenceClassification.from_pretrained(
    "beomi/KcELECTRA-base",
    num_labels=len(labels) # 감정 개수
    # problem_type="multi_label_classification" # Multi-class classification이므로 제거
)

# 3. 트레이너 및 학습
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    report_to="none" # W&B 로깅 비활성화
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer # Removed as it's handled by data_collator now
)

trainer.train()
trainer.save_model('./kcelectra-emotion-kmre')


Map:   0%|          | 0/119995 [00:00<?, ? examples/s]

Map:   0%|          | 0/49997 [00:00<?, ? examples/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1866255818.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.201100,1.157468
2,1.026100,1.075431
3,0.912200,1.077371


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Google Drive에 저장할 경로 설정
save_path = '/content/drive/MyDrive/kcelectra-emotion-kmre-model'

# 모델 저장
trainer.save_model(save_path)

# 토크나이저 저장
tokenizer.save_pretrained(save_path)

print(f"모델과 토크나이저가 '{save_path}'에 성공적으로 저장되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
모델과 토크나이저가 '/content/drive/MyDrive/kcelectra-emotion-kmre-model'에 성공적으로 저장되었습니다.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 2. Define the labels list
labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']

# 3. Create an id2label dictionary
id2label = {i: label for i, label in enumerate(labels)}

# Define the path to the saved model in Google Drive
save_path = '/content/drive/MyDrive/kcelectra-emotion-kmre-model'

# 4. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_path)
print("Tokenizer loaded successfully.")

# 5. Load the model
model = AutoModelForSequenceClassification.from_pretrained(
    save_path,
    num_labels=len(labels),
    id2label=id2label
)
print("Model loaded successfully.")

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/kcelectra-emotion-kmre-model'. Use `repo_type` argument if needed.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 2. Define the labels list
labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']

# 3. Create an id2label dictionary
id2label = {i: label for i, label in enumerate(labels)}

# Define the path to the saved model in Google Drive
save_path = '/content/drive/MyDrive/kcelectra-emotion-kmre-model'

# 4. Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_path)
print("Tokenizer loaded successfully.")

# 5. Load the model
model = AutoModelForSequenceClassification.from_pretrained(
    save_path,
    num_labels=len(labels),
    id2label=id2label
)
print("Model loaded successfully.")